In [93]:
import pandas as pd
import numpy as np
ONLY_2016 = True

In [94]:
isa_courses_2016 = pd.read_csv('./data/ISA_romain/raph_coursecode_matching.csv')
isa_courses_2016.head()

,Unnamed: 0,SubjectName,SubjectID,SectionName,CourseCode,YearName
0,0,Histoire de l'architecture I,1773962,Architecture,AR-123,2015-2016
1,1,Stéréotomie,1769400055,Architecture,AR-211,2015-2016
2,2,Introduction aux outils CAO en architecture,1888400347,Architecture,AR-219,2015-2016
3,3,Théorie de l'architecture III,1776787,Architecture,AR-221,2015-2016
4,4,Histoire de l'architecture III,1773972,Architecture,AR-223,2015-2016


Let's use the data from ISA to filter Kshitij's data and keep only the courses from 2015-2016, as we're not interested in old courses:

In [95]:
# Note that this contains each course twice, once in english and once in french
max_courses_full = pd.read_csv('./data/Max/courses.csv')[['Code', 'Title']]
max_courses_full.head()

,Code,Title
0,AR-101,Studio BA1
1,AR-101,Théorie et critique du projet BA1
2,AR-102,Studio BA2
3,AR-102,Théorie et critique du projet BA2
4,AR-111,Figuration graphique I


In [96]:
# Baseline edges scraped by Max
baseline_edges = pd.read_csv('./data/Max/edges/pre_obl.csv')[['Source', 'Target']]
baseline_edges.head()

,Source,Target
0,AR-112,AR-111
1,AR-211,MATH-121
2,AR-219,MATH-126
3,AR-231,CIVIL-122
4,AR-239,CIVIL-122


# Match Max's data with Kshitij's data

The problem here is that Max's data was scraped from the web, and thus the courses do not have a subject ID. Also, the data was scraped in 2017, but the data from ISA that Kshitij used only goes until 2016.
Therefore, the goal is to match the data by course code and/or course name as best as possible, considering that course codes/names can change over the years. Since this might create inconsistencies, I decided to process as follow: 
- The baseline (Max's scraped data) should stay true no matter what
- Add as much edges from Kshitij's data as you can. It's fine if some are missing
- It's okay if Kshitij's data has mistakes/inconsistencies, but try to avoid them as much as possible

### Data Exploration: Kshitij's data

In [97]:
courses_ksh = pd.read_csv('./data/Kshitij_new/isa-subject-id-name-code-hash.csv')
courses_ksh.head()

,SubjectID,SubjectName,AssociatedCourseCodes
0,1179233,"Analyse I, II (allemand)",NaN
1,1179240,Algèbre linéaire I,MATH-110; MATH-110(b); MATH-112; MATH-112(a); ...
2,1179245,Analyse numérique,MATH-150; MATH-250; MATH-251(a); MATH-251(b); ...
3,1179253,Plan BA/MA,NaN
4,1186202,Algèbre linéaire II,MATH-115; MATH-117; MATH-117(a); MATH-117(b)


In [98]:
print("There are " + str(courses_ksh.AssociatedCourseCodes.isnull().sum()) + "/" + str(len(courses_ksh)) + " courses with no codes")

There are 3278/6365 courses with no codes


In [99]:
grades_cor = pd.read_csv('./data/Kshitij_new/correlation-subject-pair.csv')[['sub1', 'sub2', 'cor1', 'pval1', 'cor2', 'pval2', 'sameSemester']]
grades_cor.head()

,sub1,sub2,cor1,pval1,cor2,pval2,sameSemester
0,1179240,1179245,0.479456,1.053786e-71,0.578663,1.057758e-13,0
1,1179240,1186202,0.638191,2.904612e-168,0.562914,1.925217e-17,2
2,1179240,1705532,0.491199,3.313850e-08,0.534409,7.964026e-13,594
3,1179240,1705538,0.601198,1.168207e-63,0.611785,6.559217e-15,0
4,1179240,1705590,0.488534,1.368027e-23,0.444710,6.189562e-12,1186


In [100]:
# Let's see how many edges we would have to drop if we drop all courses with no courseCode
courses_ksh_no_null = set(courses_ksh.dropna(subset=['AssociatedCourseCodes']).SubjectID.values)
#courses_ksh_no_null = courses_ksh[pd.notnull(courses_ksh['AssociatedCourseCodes'])]
print(len(courses_ksh_no_null))
n_drops_edges = 0
for row in grades_cor.values:
    if row[0] not in courses_ksh_no_null or row[1] not in courses_ksh_no_null:
        n_drops_edges += 1
print("We would drop " + str(n_drops_edges) + "/" + str(len(grades_cor)) + " edges if we drop all courses with no codes")

3087
We would drop 0/5828 edges if we drop all courses with no codes


It seems there is no edge from/to courses that have no course code. Thus, we can discard all the courses that have no code. \o/

In [101]:
# Drop the data with no course code
courses_ksh.dropna(subset=['AssociatedCourseCodes'], inplace=True)

### Filter Kshitij's courses to keep only those from 2016

In [102]:
courses_ksh_2016 = courses_ksh[courses_ksh['SubjectID'].isin(isa_courses_2016['SubjectID'])]
courses_ksh_2016.head()

,SubjectID,SubjectName,AssociatedCourseCodes
2,1179245,Analyse numérique,MATH-150; MATH-250; MATH-251(a); MATH-251(b); ...
14,1705590,Analyse I,MATH-100; MATH-100(a); MATH-100(b); MATH-101(a...
18,1748172,Analyse I (allemand),MATH-101(de); MATH-104
19,1770632,Barrages et ouvrages hydrauliques annexes,CIVIL-411
22,1770678,Applied molecular quantum chemistry,CH-451


In [103]:
print('there are ' + str(courses_ksh_2016.shape[0]) + '/' + str(courses_ksh.shape[0]) + " subjects from 2016 in khitij's data")

there are 764/3087 subjects from 2016 in khitij's data


In [104]:
if ONLY_2016:
    courses_ksh_final = courses_ksh_2016
else:
    courses_ksh_final = courses_ksh

- The unique courses will be courses_ksh_2016
- The unique identifier will be the subject id
- Process todo next:
    - asssign the corresponding subjectID to each max_courses
    - use that to translate the max_edges from code->code to subID->subID
    - Now we have the baseline edges
    - Then add the edges from Kshitij and get their directions with the grades file
    
## For each course of Max, find a subject ID
### Then, translate the baseline edges from (courseCode) <-> (courseCode) to (subID) <-> (subID)
We can either matche the CC to the subIDs by course name, or by course code. I chose to match by course name, because it prevents mistakes (course codes can change over the years) due to the fact that both datasets are not from the same year

In [105]:
# join with course names
temp = courses_ksh_final
temp.index = courses_ksh_final.SubjectName
join_courses = max_courses_full.join(courses_ksh_final, on='Title')
join_courses = join_courses[pd.notnull(join_courses['SubjectName'])]
join_courses.drop(columns=['SubjectName', 'Title', 'AssociatedCourseCodes'], inplace=True)

# make a dict max_code -> subjectID
code_to_id = join_courses.set_index(join_courses['Code']).drop(columns=['Code']).to_dict()['SubjectID']

# translate the baseline edges
baseline_edges_final = baseline_edges[baseline_edges['Source'].isin(code_to_id) & baseline_edges['Target'].isin(code_to_id)].replace(code_to_id)
baseline_edges_final.head()



,Source,Target
9,7.147139e+07,2.381526e+07
12,3.510539e+08,1.772025e+06
13,3.522523e+08,1.772095e+06
19,1.678983e+09,1.678983e+09
21,1.773437e+06,1.772095e+06


In [106]:
count = 0
for row in baseline_edges.values:
    if row[0] in code_to_id and row[1] in code_to_id:
        count += 1
print('We will keep ' + str(count) + '/' + str(baseline_edges.shape[0]) + ' baseline edges')

We will keep 60/226 baseline edges


## Process Kshitij's edges (Solution 1)
The first solution is to take from the LDA edges all the pair of courses that appear in the grades correlations, so that we can infer their direction for the correlation. However, the two files only have ~100 common edges, which is really not enough.

In [107]:
# build dict name -> id
name_to_id = dict()
for row in courses_ksh_final.values:
    name_to_id[row[1]]= row[0]

### Import edges and find subjectIDs of their endpoints

Choose if you want to make the match the courses by ID or by name:

In [108]:
# import the edges
lda_edges = pd.read_csv('./data/Kshitij_old/edge_list_inf1_lda.csv', error_bad_lines=False)

# match with names
'''
lda_edges = lda_edges[lda_edges['course1'].isin(courses_ksh_final.SubjectName) & lda_edges['course2'].isin(courses_ksh_final.SubjectName)]
# rename the lda edges
lda_edges['ID1'] = lda_edges.course1.replace(name_to_id)
lda_edges['ID2'] = lda_edges.course2.replace(name_to_id)
'''


# or match with IDs
lda_edges = lda_edges[lda_edges['code1'].isin(code_to_id) & lda_edges['code2'].isin(code_to_id)]
# rename the lda edges
lda_edges['ID1'] = lda_edges.code1.replace(code_to_id)
lda_edges['ID2'] = lda_edges.code2.replace(code_to_id)

b'Skipping line 5142: expected 4 fields, saw 5\nSkipping line 5146: expected 4 fields, saw 6\nSkipping line 5518: expected 4 fields, saw 8\nSkipping line 6462: expected 4 fields, saw 5\nSkipping line 6666: expected 4 fields, saw 6\nSkipping line 7607: expected 4 fields, saw 6\nSkipping line 8660: expected 4 fields, saw 5\n'


In [109]:
# drop unnececary columns
lda_edges_final = lda_edges.drop(['code1', 'code2', 'course1', 'course2'], axis=1)
lda_edges_final.head()
lda_edges_final.shape

(1593, 2)

### Match edges and grades correlations

In [110]:
# create the set of edges by taking each edge of the lda_edge_set if it has a correspondance in the grade set
# if there is a correspondance, use it to find the direction of the edge
'''
lda_edges_final_dict = dict()
i = 0
for edge in lda_edges_final.values:
    i += 1
    if i % 100 == 0:
        print(str(int(100*i/lda_edges_final.shape[0])) + '%')
    for row in grades_cor.values:
        if (int(edge[0]) == int(row[0]) and int(edge[1]) == int(row[1])) or (int(edge[0]) == int(row[1]) and int(edge[1]) == int(row[0])):
            if row[2] > row[4]:
                lda_edges_final_dict[row[0]] = row[1]
            else:
                lda_edges_final_dict[row[1]] = row[0]
'''

"\nlda_edges_final_dict = dict()\ni = 0\nfor edge in lda_edges_final.values:\n    i += 1\n    if i % 100 == 0:\n        print(str(int(100*i/lda_edges_final.shape[0])) + '%')\n    for row in grades_cor.values:\n        if (int(edge[0]) == int(row[0]) and int(edge[1]) == int(row[1])) or (int(edge[0]) == int(row[1]) and int(edge[1]) == int(row[0])):\n            if row[2] > row[4]:\n                lda_edges_final_dict[row[0]] = row[1]\n            else:\n                lda_edges_final_dict[row[1]] = row[0]\n"

In [111]:
#len(lda_edges_final_dict)
#lda_edges_final_dict

This is not satisfying, as the edges from Kshitij (PSLA, LDA, ...) are completly different from the 'grades corelations' pairs. Thus we only get directions for ~100 edges.
## The other solution: directly use the grades corelations as edges

In [112]:
edges_grades = pd.DataFrame()
edges_grades['Source'] = grades_cor.apply(lambda x: x[0] if x[2] > x[4] else x[1], axis=1)
edges_grades['Target'] = grades_cor.apply(lambda x: x[0] if x[2] <= x[4] else x[1], axis=1)
edges_grades['cor'] = grades_cor.apply(lambda x: max(x[2], x[4]), axis=1)
edges_grades['sameSemester'] = grades_cor['sameSemester']

In [113]:
edges_grades.head()

,Source,Target,cor,sameSemester
0,1179245.0,1179240.0,0.578663,0
1,1179240.0,1186202.0,0.638191,2
2,1705532.0,1179240.0,0.534409,594
3,1705538.0,1179240.0,0.611785,0
4,1179240.0,1705590.0,0.488534,1186


In [114]:
print(edges_grades.shape)
print(edges_grades[edges_grades.cor > 0.5].shape)

(5828, 4)
(883, 4)


### Keep only the edges from/to 2016 courses

In [115]:
# Keep only edges that have both endpoints in the courses' list and have a correlation > 0

correlation_threshold = 0

if ONLY_2016:
    edges_grades_final = edges_grades[edges_grades['Source'].isin(courses_ksh_2016.SubjectID) & edges_grades['Target'].isin(courses_ksh_2016.SubjectID)]
    edges_grades_final = edges_grades_final[edges_grades_final.cor > correlation_threshold][['Source', 'Target']]
else:
    edges_grades_final = edges_grades[edges_grades['Source'].isin(courses_ksh.SubjectID) & edges_grades['Target'].isin(courses_ksh.SubjectID)]
    edges_grades_final = edges_grades_final[edges_grades_final.cor > correlation_threshold][['Source', 'Target']]

print(edges_grades_final.shape)

(1255, 2)


## Final Processing and Exporting

In [116]:
courses = courses_ksh_final.reset_index(drop=True).rename({'SubjectID':'Id'}, axis=1)
edges = pd.concat([baseline_edges_final, edges_grades_final]).astype(int)

In [117]:
# Sanity check
print(edges['Source'].isin(courses.Id).all())
print(edges['Target'].isin(courses.Id).all())

True
True


In [118]:
courses.to_csv('./data/output/courses_grades_2016.csv', index=False)
edges.to_csv('./data/output/edges_grades_2016.csv', index=False)

## Third solution: Export the LDA edges and ignore the direction

In [119]:
lda_edges_final.rename({'ID1':'Source', 'ID2':'Target'}, axis=1).astype(int).to_csv('./data/output/edges_LDA_undir_2016.csv', index=False)